<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/WOOK/GraphCodeBERT_1_N_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import pandas as pd

In [4]:
#df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype5.csv')
df = pd.read_csv('/content/drive/MyDrive/Labels_Safe_MSR+Juliet_no_random+106columns.csv의 사본')

In [5]:
df

,Unnamed: 0.1,Unnamed: 0,CWE ID,lang,vul,code,labels,chmod(),assert(),atoi(),...,array_index_vulnerability,int,short,long,long long,unsigned int,unsigned short,unsigned long,unsigned long long,line_count
0,0,115.0,safe,c,0,"void func(fz_context *ctx, fz_color_converter ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,1,116.0,safe,c,0,"void func(fz_context *ctx, fz_color_converter ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,2,117.0,safe,c,0,"void func(fz_context *ctx, const fz_colorspace...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,3,118.0,safe,c,0,static inline void func(unsigned char *restric...,0,0,0,0,...,0,8,0,0,0,8,0,0,0,113
4,4,119.0,safe,c,0,"void func(const fz_colorspace *cs, const float...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16396,16396,NaN,CWE-416,cpp,1,void func() {\n\t wchar_t * data; /* Ini...,8,0,0,0,...,0,0,0,0,0,0,0,0,0,7
16397,16397,NaN,CWE-416,cpp,1,void func() {\n\t wchar_t * data; /* Ini...,8,0,0,0,...,0,0,0,0,0,0,0,0,0,7
16398,16398,NaN,CWE-121,c,1,void func() {\n\t wchar_t * data; other_...,4,0,0,0,...,0,0,0,0,0,0,0,0,0,7
16399,16399,NaN,CWE-121,c,1,static void func(wchar_t * data) {\n\t {\n...,4,0,0,0,...,0,0,0,0,0,0,0,0,0,7


In [6]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'CWE ID', 'lang', 'vul', 'code', 'labels',
       'chmod()', 'assert()', 'atoi()',
       ...
       'array_index_vulnerability', 'int', 'short', 'long', 'long long',
       'unsigned int', 'unsigned short', 'unsigned long', 'unsigned long long',
       'line_count'],
      dtype='object', length=107)

In [7]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/graphcodebert-base", num_labels = 11)
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [9]:

df = df.drop(columns = "vul")

In [10]:
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from datasets import load_dataset

def preprocess_code(df):
    # code 컬럼 전처리
    df['code'] = df['code'].apply(lambda x: re.sub(r'/\*.*?\*/', '', x))  # 멀티 라인 주석 제거
    df['code'] = df['code'].apply(lambda x: re.sub(r'//.*?\n', '', x))   # 싱글 라인 주석 제거
    df['code'] = df['code'].apply(lambda x: re.sub(r'#include <.*?>\n', '', x))  # angle brackets를 사용하는 include 제거
    df['code'] = df['code'].apply(lambda x: re.sub(r'#include \".*?\"\n', '', x))  # double quotes를 사용하는 include 제거
    df['code'] = df['code'].apply(lambda x: re.sub(r'#define .*?\n', '', x))  # 매크로 정의 제거
    df['code'] = df['code'].apply(lambda x: re.sub(r'[\t ]+', ' ', x))  # 탭과 여러 공백을 하나의 공백으로
    df['code'] = df['code'].apply(lambda x: re.sub(r'\n\s*\n', '\n', x))  # 여러 줄바꿈을 하나로
    return df
def tokenized(examples):
  return tokenizer(examples['code'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [11]:
# 데이터 전처리
preprocessed_df = preprocess_code(df)

# "code","labels" 컬럼을 제외한 열 선택
columns_to_encode = [col for col in preprocessed_df.columns if col not in ["code", "labels"]]

# 원핫인코딩 수행
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(preprocessed_df[columns_to_encode])

# 인코딩된 데이터를 데이터프레임으로 변환
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(columns_to_encode))

# "code" 컬럼과 인코딩된 데이터 결합
result_df = pd.concat([preprocessed_df[["code", "labels"]], encoded_df], axis=1)

# 데이터프레임을 CSV 파일로 저장
result_df.to_csv("preprocessed_data.csv", index=False)


In [12]:
# 데이터셋 로드 및 전처리
dataset = load_dataset("csv", data_files="preprocessed_data.csv")['train']

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
from sklearn.model_selection import KFold
import torch
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback 설정
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5,  # 성능 개선이 3 에폭 동안 관찰되지 않으면 중단
    early_stopping_threshold=0.01  # 개선된 것으로 간주하기 위한 최소 변경량
)

# K-Fold 교차 검증 설정
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# 데이터셋 분할 및 토큰화
tokenized_dataset = dataset.map(tokenized, remove_columns=['code'], batched=True)




Map:   0%|          | 0/16401 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics


In [18]:
# 각 fold에 대해 모델 훈련 및 평가 실행
for fold, (train_index, test_index) in enumerate(kf.split(tokenized_dataset)):
    print(f"Training fold {fold+1}/{k}")
    train_dataset = tokenized_dataset.select(train_index)
    test_dataset = tokenized_dataset.select(test_index)

    # 모델 초기화
    model = AutoModelForSequenceClassification.from_pretrained("microsoft/graphcodebert-base", num_labels=11)

    # 옵티마이저와 스케줄러 초기화
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.99), eps=1e-8, weight_decay=0.1)
    total_steps = len(train_dataset) // 16 * 7  # per_device_train_batch_size * num_train_epochs
    scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)

    # Trainer 설정
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        num_train_epochs=100,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        logging_dir=f'./logs_fold_{fold}'
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        optimizers=(optimizer, scheduler),
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
    )

Training fold 1/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training fold 2/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training fold 3/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training fold 4/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training fold 5/5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9,F1 Class 10
1,1.604600,0.680263,0.750915,0.750915,0.740386,0.685517,0.750915,0.778962,0.614072,0.894737,0.877086,0.879656,0.915541,0.409962,0.840426,0.780669,0.038710,0.510870
2,0.617800,0.547568,0.805793,0.805793,0.799897,0.753307,0.805793,0.878738,0.684318,0.924433,0.886931,0.883721,0.936791,0.480392,0.876147,0.783172,0.382775,0.568966
3,0.498800,0.525038,0.821951,0.821951,0.823324,0.792068,0.821951,0.871545,0.707434,0.924433,0.888587,0.879499,0.957411,0.547414,0.932367,0.824742,0.532847,0.646465
4,0.345300,0.552557,0.832622,0.832622,0.830846,0.798530,0.832622,0.887584,0.719921,0.924433,0.894118,0.886396,0.952055,0.583784,0.929782,0.813115,0.541667,0.650980


Epoch,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9,F1 Class 10
1,1.604600,0.680263,0.750915,0.750915,0.740386,0.685517,0.750915,0.778962,0.614072,0.894737,0.877086,0.879656,0.915541,0.409962,0.840426,0.780669,0.038710,0.510870
2,0.617800,0.547568,0.805793,0.805793,0.799897,0.753307,0.805793,0.878738,0.684318,0.924433,0.886931,0.883721,0.936791,0.480392,0.876147,0.783172,0.382775,0.568966
3,0.498800,0.525038,0.821951,0.821951,0.823324,0.792068,0.821951,0.871545,0.707434,0.924433,0.888587,0.879499,0.957411,0.547414,0.932367,0.824742,0.532847,0.646465
4,0.345300,0.552557,0.832622,0.832622,0.830846,0.798530,0.832622,0.887584,0.719921,0.924433,0.894118,0.886396,0.952055,0.583784,0.929782,0.813115,0.541667,0.650980
5,0.256600,0.567736,0.827744,0.827744,0.826100,0.796572,0.827744,0.874194,0.728070,0.924433,0.863976,0.880707,0.951747,0.572178,0.924171,0.797508,0.602740,0.642570
6,0.212900,0.589058,0.829878,0.829878,0.828903,0.800204,0.829878,0.879675,0.729670,0.924433,0.864865,0.870968,0.953333,0.601432,0.920930,0.823529,0.585185,0.648221
7,0.177800,0.594214,0.831402,0.831402,0.830607,0.801795,0.831402,0.881633,0.733260,0.924433,0.865762,0.872993,0.953333,0.609929,0.920930,0.826230,0.583026,0.648221
8,0.171200,0.603991,0.828963,0.828963,0.827993,0.800050,0.828963,0.879804,0.728249,0.924433,0.860947,0.862222,0.953333,0.599057,0.925581,0.827815,0.577947,0.661157
9,0.175000,0.608315,0.830793,0.830793,0.830056,0.801388,0.830793,0.878168,0.734554,0.924433,0.874644,0.869301,0.951586,0.601504,0.919540,0.815534,0.589404,0.656604
10,0.184700,0.677312,0.830793,0.830793,0.831552,0.802777,0.830793,0.879085,0.729167,0.924433,0.870871,0.891462,0.956522,0.585859,0.927059,0.823129,0.573643,0.669323


In [ ]:
trainer.save_model('model_final_train_feature')

In [ ]:
torch.cuda.empty_cache()
gc.collect()